# Triangle Quickstart

This tutorial covers the basics of using the **Triangle workflow** - agent-workshop's primary feature for automated PR generation from GitHub issues.

## What is Triangle?

Triangle automates the software development cycle:

```
Issue → Code → PR → Review → Merge
         ↑                    ↓
         └──── Feedback ──────┘
```

Key features:
- **Human-gated checkpoints** - Workflow pauses for review before merge
- **AI code review** - Greptile integration for automated feedback
- **Auto-fix comments** - PRCommentProcessor addresses review feedback
- **Per-project config** - Customizable via `.triangle.toml`

## Prerequisites

Before using Triangle, you need:

1. **agent-workshop installed**
   ```bash
   pip install agent-workshop
   ```

2. **GitHub CLI configured**
   ```bash
   gh auth login
   ```

3. **Environment variables** (in your project's `.env`):
   ```bash
   AGENT_WORKSHOP_ENV=development
   CLAUDE_SDK_ENABLED=true
   ```

In [1]:
# Verify installation
import subprocess

# Check triangle CLI is available
result = subprocess.run(["triangle", "--help"], capture_output=True, text=True)
if result.returncode == 0:
    print("✅ Triangle CLI is installed")
    print(result.stdout[:500])
else:
    print("❌ Triangle CLI not found. Run: pip install agent-workshop")

✅ Triangle CLI is installed
Usage: triangle [OPTIONS] COMMAND [ARGS]...

  Triangle workflow control for human-gated automation.

  The triangle workflow automates software development tasks with human
  approval checkpoints at key decision points.

  Commands:     start    - Start a new triangle workflow for an issue
  status   - Show workflow status (all or specific)     approve  - Approve a
  human checkpoint to resume workflow     list     - List all active/pending
  workflows     cancel   - Cancel a running workflow




## Basic CLI Commands

Triangle provides three main commands:

### 1. Start a Workflow

```bash
triangle start --issue 42 --repo owner/repo
```

This will:
- Fetch and parse the GitHub issue
- Create a git worktree for isolated development
- Generate code to implement the requirements
- Run verification (lint, type check)
- Create a draft PR
- **STOP** for human review

### 2. Check Status

```bash
triangle status
```

Shows all active workflows and their current state.

### 3. Approve and Continue

```bash
triangle approve owner-repo-issue-42
```

After reviewing the PR, approve to:
- Process any review comments
- Apply fixes automatically
- Complete the workflow

## Workflow Steps Explained

### Step 1: Parse Issue

Triangle fetches the issue from GitHub and uses an LLM to extract:
- Requirements
- Acceptance criteria
- Files to create/modify
- Complexity estimate

### Step 2: Setup Worktree

Creates an isolated git worktree so you can run multiple workflows in parallel without conflicts.

### Step 3: Generate Code

The LLM generates code based on:
- Issue requirements
- Project style guidelines (from `.triangle.toml`)
- Previous verification errors (if retrying)

### Step 4: Verify Code

Runs tiered verification:
1. **Syntax** - Python parses correctly
2. **Lint** - ruff/flake8 passes
3. **Type** - pyright/mypy passes
4. **Test** - pytest passes

If verification fails, Triangle retries code generation (up to 3 times).

### Step 5: Create PR

Creates a draft PR with:
- Summary of changes
- Link to the issue
- Checklist for review

### Step 6: Human Checkpoint

**The workflow STOPS here.** This is the human-gated checkpoint.

You (and optionally Greptile AI) review the PR. When ready, run `triangle approve`.

### Step 7: Process Comments

After approval, PRCommentProcessor:
- Fetches review comments
- Analyzes each comment
- Generates fixes
- Commits changes

### Step 8: Complete

Workflow completes. The PR is ready for final merge.

## Example: End-to-End Demo

Here's a complete example workflow.

### 1. Create a Test Issue

```bash
gh issue create --repo owner/repo \
  --title "Add hello world function" \
  --body "Create a function that returns 'Hello, World!'"
```

### 2. Start the Workflow

```bash
triangle start --issue 1 --repo owner/repo
```

Output:
```
🔺 Starting Triangle workflow for issue #1
📋 Parsing issue...
🌳 Setting up worktree...
💻 Generating code...
✅ Verification passed
📝 Creating PR...
⏸️  Workflow paused at checkpoint

PR created: https://github.com/owner/repo/pull/2
Thread ID: owner-repo-issue-1

Review the PR, then run:
  triangle approve owner-repo-issue-1
```

### 3. Review the PR

Go to the PR URL and review the generated code. Add comments if needed.

### 4. Approve and Complete

```bash
triangle approve owner-repo-issue-1
```

Output:
```
🔺 Resuming workflow owner-repo-issue-1
💬 Processing 2 review comments...
🔧 Applying fixes...
✅ Workflow complete!
```

## Programmatic Usage

You can also use Triangle components directly in Python:

In [2]:
# Example: Using IssueToPR programmatically
from agent_workshop import Config
from agent_workshop.agents.software_dev import IssueToPR, make_thread_id
from agent_workshop.utils.persistence import get_checkpointer

# Create workflow with checkpointing
config = Config()
checkpointer = get_checkpointer()

workflow = IssueToPR(
    config=config,
    checkpointer=checkpointer
)

print(f"Workflow nodes: {list(workflow.graph.nodes.keys())}")
print(f"Thread ID format: {make_thread_id('owner/repo', 42)}")

Workflow nodes: ['__start__', 'parse_issue', 'setup_worktree', 'generate_code', 'verify_code', 'create_pr', 'await_review']
Thread ID format: owner-repo-issue-42


## Next Steps

- **[09_triangle_configuration.ipynb](./09_triangle_configuration.ipynb)** - Customize Triangle with `.triangle.toml`
- **[04_pr_pipeline.ipynb](./04_pr_pipeline.ipynb)** - Learn about PRPipeline for code review
- **[README](../../README.md)** - Full documentation